In [ ]:
import pathlib
import time
import networkx as nx
import itertools
import numpy as np
import os

def parse_problem(line):
    problem_str, solution_str = line.split('output')
    
    G = nx.Graph()
    
    problem_i = iter(problem_str.strip().split(' '))
    for x in problem_i:
        y = next(problem_i)
        pos = np.array([float(x), float(y)])
        G.add_node(len(G.nodes), pos=pos)

    solution = [int(x) - 1 for x in solution_str.strip().split(' ')]
    solution_edges = [e for e in zip(solution[:-1], solution[1:])]

    for i, j in itertools.combinations(G.nodes, 2):
        # length of edge
        d = np.linalg.norm(G.nodes[i]['pos'] - G.nodes[j]['pos'])
        # dist from depot to nodes
        di = np.linalg.norm(G.nodes[i]['pos'] - G.nodes[0]['pos'])
        dj = np.linalg.norm(G.nodes[j]['pos'] - G.nodes[0]['pos'])
        # avg dist from depot to nodes
        dij = (di + dj)/2
        in_sol = (i, j) in solution_edges or (j, i) in solution_edges
        G.add_edge(i, j, dist=d, dist_to_depot=dij, in_sol=in_sol)

    k = 5
    nx.set_edge_attributes(G, False, 'nn')
    for source, paths in nx.shortest_path_length(G, weight='dist'):
        nn = sorted(paths.items(), key=lambda x: x[1])
        for dest, dist in nn[1:k + 1]:
            G.edges[source, dest]['nn'] = True
        
    return G

data_path = pathlib.Path('.')
os.makedirs('data', exist_ok=True)
with open(data_path / 'tsp30_concorde.txt') as data_file:
    for line_i, line in enumerate(data_file):
        G = parse_problem(line)
        nx.write_gpickle(G, path=f'data/g{line_i}.pkl')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

In [ ]:
G = nx.read_gpickle('data/g9999.pkl')

plt.figure()
pos = nx.get_node_attributes(G, 'pos')
edge_colors = [float(v) for k, v in nx.get_edge_attributes(G, 'nn').items()]
nx.draw(G, pos, edge_color=edge_colors, edge_cmap=plt.cm.viridis)

In [ ]:
plt.figure()

pos = nx.get_node_attributes(G, 'pos')
edge_list = [e for e in G.edges if G.edges[e]['in_sol']]
nx.draw(G, pos, edgelist=edge_list, with_labels=True)